# Process all measurement data

In [5]:
base <- "/data/reiner/jss-experiments/execution-2/"

for (experiment in c(1,2,3,4,5,6,7,8,9,
                     10,11,12,13,14,15,16,17,18,19,
                     20,30,40,50,60,70,80,90,95,
                     100,110,120,130,140,150,
                     200,300,400,
                     500,550,560,570,580,590,
                     600,
                     700,710,720,730,740,750,
                     800,
                     900,950,960,970,980,990,
                     1000,1010,1020,1030,1040,1050,
                     2000,3000,4000,5000,6000,7000,8000,9000,
                     10000)) {
    print(sprintf("Experiment %d", experiment))
    experimentDirectory <- sprintf("%s/simulated-account-services-%s", base, experiment)
    if (file.exists(experimentDirectory)) {
        data <- ""
        for (loop in 0:200) {
            measurementFile <- sprintf("%s/simulated-account-services-%s/%d/performance-results/execution-%d.csv",
                                       base, experiment, loop, loop)
            if (file.exists(measurementFile)) {
                if (file.size(measurementFile) > 0) {
                    #print(sprintf("Reading %s ...", measurementFile))
                    input <- read.csv(measurementFile)
                    
                    if (typeof(data) == "character") {
                        data <- input
                    } else {
                        data <- rbind(data, input)
                    }
                } else {
                    print(sprintf("File %s is empty.", measurementFile))
                }
            }    
        }

        print(sprintf("Input records %d", length(data[,1])))
        
        summaryFile <- sprintf("data/summary/simulated-account-services-%05d.csv", experiment)

        label.vals <- c()
        mean.vals <- c()
        median.vals <- c()
        var.vals <- c()
        ql.vals <- c()
        qh.vals <- c()
        q1.vals <- c()
        q2.vals <- c()
        q3.vals <- c()
        q4.vals <- c()
        q6.vals <- c()
        q7.vals <- c()
        q8.vals <- c()
        q9.vals <- c()
        
        unit.conversion <- 1000 * 1000

        cleaned <- subset(data, !is.na(total.processing))
              
        print(sprintf("Proper records %d", length(cleaned[,1])))

        i <- 1
        for (label in colnames(data)) {
            label.vals[[i]] <- gsub("[.]", " ",label)
            columnVectorData <- cleaned[,i] / unit.conversion

            median.vals[[i]] <- median(columnVectorData, na.rm=TRUE)
            mean.vals[[i]] <- mean(columnVectorData, na.rm=TRUE)
            var.vals[[i]] <- var(columnVectorData, na.rm=TRUE)
            quartiles <- quantile(columnVectorData, probs=c(0.25,0.75), names=FALSE, na.rm=TRUE)
            ql.vals[[i]] <- quartiles[1]
            qh.vals[[i]] <- quartiles[2]
            quantiles <- quantile(columnVectorData, probs=c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9), names=FALSE, na.rm=TRUE)
            q1.vals[[i]] <- quantiles[1]
            q2.vals[[i]] <- quantiles[2]
            q3.vals[[i]] <- quantiles[3]
            q4.vals[[i]] <- quantiles[4]
            q6.vals[[i]] <- quantiles[6]
            q7.vals[[i]] <- quantiles[7]
            q8.vals[[i]] <- quantiles[8]
            q9.vals[[i]] <- quantiles[9]

            i <- i + 1
        }

        result <- data.frame("measurement" = label.vals,
            "mean" = mean.vals,
            "median" = median.vals, 
            "variance" = var.vals,
            "ql" = ql.vals,
            "qh" = qh.vals, 
            "q1" = q1.vals,
            "q2" = q2.vals,
            "q3" = q3.vals,
            "q4" = q4.vals,
            "q6" = q6.vals,
            "q7" = q7.vals,
            "q8" = q8.vals,
            "q9" = q9.vals)

        print(result)
        write.csv(result, summaryFile)
    } else {
        print(sprintf("Missing data for %d", experiment))
    }
}

[1] "Experiment 1"
[1] "Input records 231448"
[1] "Proper records 231447"
         measurement        mean      median     variance          ql
1               date 53381.38830 41172.35353 1.680128e+09 18326.74386
2 configuration time  1378.01544  1240.97234 6.518072e+05  1077.95666
3       model update    38.15657    39.95813 1.299926e+03    39.76857
4      reaction time 80313.58807 89407.51346 7.781512e+08 58434.29923
5     privacy update   144.88215   145.47989 1.072592e+04   122.21633
6   total processing 80313.57562 89407.51111 7.781518e+08 58434.29644
            qh           q1          q2          q3         q4         q6
1  86811.86776 7.008969e+03 14388.61043 21925.60809 30850.7334 57468.8411
2   1438.42266 9.292469e+02  1034.96629  1118.77137  1178.0647  1321.8058
3     40.09573 9.565106e-01    39.42188    39.87557    39.9276    39.9851
4 103529.22456 4.121175e+04 52510.17881 64474.57905 76901.3971 95911.4243
5    164.39354 5.681608e+01    99.43048   124.77667   142.0028   1